# Primeras dudas:

## Preprocesamiento de Datos:

* Manejo de Valores Faltantes (fillna)
* Codificación One-Hot (pd.get_dummies)
* Normalización (Escalado Estándar - StandardScaler)

# Manejo de Valores Faltantes

En tu caso, para la columna 'Type 2', el concepto de un Pokémon sin un segundo tipo es una "ausencia" natural de un valor, no un error de datos. Rellenarlo con la constante 'None' es una estrategia apropiada porque:

Permite que pd.get_dummies() cree una columna Type2_None.
Trata a los Pokémon de un solo tipo como una categoría distinta y comparable en el espacio de características.

In [1]:
import pandas as pd
# Suponemos que 'df' ya ha sido cargado, por ejemplo:
df = pd.read_csv('dataset/pkmn.csv') 

# Antes del manejo de faltantes, veamos cómo se ve la columna 'Type 2'
# Puedes ejecutar esto en tu Jupyter Notebook para ver los NaN
print("Valores únicos en 'Type 2' ANTES de fillna:")
print(df['Type 2'].unique())
print("\nConteo de valores nulos en 'Type 2' ANTES de fillna:")
print(df['Type 2'].isnull().sum())

# --- Parte del código para el manejo de valores faltantes ---

# Rellenar los valores nulos (NaN) en la columna 'Type 2' con la cadena 'None'
df['Type 2'] = df['Type 2'].fillna('None')

# --- Verificamos el resultado después de fillna ---
print("\nValores únicos en 'Type 2' DESPUÉS de fillna:")
print(df['Type 2'].unique())
print("\nConteo de valores nulos en 'Type 2' DESPUÉS de fillna:")
print(df['Type 2'].isnull().sum())

Valores únicos en 'Type 2' ANTES de fillna:
['Poison' nan 'Flying' 'Ground' 'Fairy' 'Grass' 'Fighting' 'Psychic'
 'Steel' 'Ice' 'Rock' 'Water' 'Electric' 'Fire' 'Dragon' 'Dark' 'Ghost'
 'Bug']

Conteo de valores nulos en 'Type 2' ANTES de fillna:
258

Valores únicos en 'Type 2' DESPUÉS de fillna:
['Poison' 'None' 'Flying' 'Ground' 'Fairy' 'Grass' 'Fighting' 'Psychic'
 'Steel' 'Ice' 'Rock' 'Water' 'Electric' 'Fire' 'Dragon' 'Dark' 'Ghost'
 'Bug']

Conteo de valores nulos en 'Type 2' DESPUÉS de fillna:
0


### # Codificación One-Hot (`pd.get_dummies()`)

**1. Teoría: ¿Qué es la Codificación One-Hot y por qué es necesaria?**

**Concepto:**

La Codificación One-Hot (también conocida como "dummy coding" o creación de variables dummy) es una técnica de preprocesamiento utilizada para convertir **variables categóricas nominales** en un formato numérico que los algoritmos de Machine Learning puedan entender y procesar.

Las variables categóricas nominales son aquellas que representan categorías sin ningún orden intrínseco o jerarquía (por ejemplo, 'Red', 'Blue', 'Green' para colores; 'Water', 'Fire', 'Grass' para tipos de Pokémon).

**El Problema con la Representación Numérica Directa:**

Imagina que intentamos asignar números directamente a los tipos de Pokémon:

* Water = 1
* Fire = 2
* Grass = 3

Si un algoritmo de ML recibiera estos números, podría interpretar que Fire (2) es de alguna manera "mayor" o "más importante" que Water (1), o que la "distancia" entre Water y Fire (1 unidad) es la misma que la distancia entre Fire y Grass (1 unidad). Esto es incorrecto y engañoso, ya que no hay una relación ordinal o de magnitud real entre estos tipos.

**La Solución de One-Hot Encoding:**

La codificación One-Hot resuelve este problema creando **nuevas columnas binarias** (que solo contienen 0s y 1s) para cada categoría única presente en la columna original. Para cada fila (observación):

* Si la observación pertenece a una categoría específica, la columna correspondiente a esa categoría tendrá un valor de `1`.
* Todas las demás nuevas columnas creadas para esa variable categórica tendrán un valor de `0`.

De esta manera, el algoritmo ve una "activación" (un 1) para la categoría a la que pertenece la observación, y una "desactivación" (un 0) para las demás. Esto elimina la implicación de orden y magnitud.

**Ejemplo Ilustrativo:**

Considera una columna `Tipo_Elemento` en un DataFrame:

| Pokémon    | Tipo_Elemento |
|:-----------|:--------------|
| Squirtle   | Water         |
| Charmander | Fire          |
| Bulbasaur  | Grass         |
| Vaporeon   | Water         |

Después de aplicar One-Hot Encoding a `Tipo_Elemento`, el DataFrame se transformaría (conceptualmente, el `Tipo_Elemento` original se eliminaría o sería la base para las nuevas columnas):

| Pokémon    | Tipo_Elemento_Water | Tipo_Elemento_Fire | Tipo_Elemento_Grass |
|:-----------|:--------------------|:-------------------|:--------------------|
| Squirtle   | 1                   | 0                  | 0                   |
| Charmander | 0                   | 1                  | 0                   |
| Bulbasaur  | 0                   | 0                  | 1                   |
| Vaporeon   | 1                   | 0                  | 0                   |

Cada fila ahora tiene una representación numérica vectorial donde solo la columna correspondiente a su tipo original es "caliente" (1).

**Consideraciones importantes:**

* **Dimensionalidad:** La codificación One-Hot puede aumentar significativamente el número de columnas en tu DataFrame si tienes variables categóricas con muchas categorías únicas. Esto se conoce como la "maldición de la dimensionalidad".
* **Colinealidad (Dummy Variable Trap):** En algunos modelos de regresión, tener todas las variables dummy puede causar colinealidad perfecta (una categoría es perfectamente predecible a partir de las otras). Esto se suele mitigar eliminando una de las columnas dummy ("k-1" categorías para "k" categorías originales). Sin embargo, para algoritmos como la similitud del coseno, esto no es un problema y a menudo se mantienen todas las columnas.

En el contexto de tu recomendador de Pokémon, la codificación One-Hot para `Type 1` y `Type 2` es fundamental porque convierte los tipos en características numéricas que pueden ser utilizadas por la similitud del coseno. Un Pokémon será más similar a la preferencia del usuario si comparten un '1' en las mismas columnas de tipo.

In [3]:
import pandas as pd
from sklearn.preprocessing import StandardScaler # Necesario para la siguiente sección, pero lo incluimos por contexto

# Suponemos que 'df' ya ha sido cargado y que 'Type 2' ya fue rellenado con 'None'
df = pd.read_csv('dataset/pkmn.csv')
df['Type 2'] = df['Type 2'].fillna('None')

# Creamos una copia del DataFrame para no modificar el original directamente
df_processed = df.copy()

# Antes de la codificación, veamos las columnas de tipos
print("Columnas ANTES de get_dummies:")
print(df_processed[['Type 1', 'Type 2']].head())

# --- Parte del código para la codificación One-Hot ---

# Aplicar One-Hot Encoding a las columnas 'Type 1' y 'Type 2'
df_processed = pd.get_dummies(
    df_processed, 
    columns=['Type 1', 'Type 2'], 
    prefix=['Type1', 'Type2'], 
    dummy_na=False # Ya manejamos los NaN con .fillna('None')
)

# --- Verificamos el resultado después de la codificación ---
# Puedes ejecutar esto en tu Jupyter Notebook para ver las nuevas columnas
print("\nColumnas DESPUÉS de get_dummies (primeras 5 filas y algunas columnas relevantes):")
print(df_processed.head()) 
# Para ver solo las columnas nuevas de tipos, podrías hacer:
type_columns = [col for col in df_processed.columns if 'Type1_' in col or 'Type2_' in col]
print(df_processed[type_columns].head())

Columnas ANTES de get_dummies:
  Type 1  Type 2
0  Grass  Poison
1  Grass  Poison
2  Grass  Poison
3   Fire    None
4   Fire    None

Columnas DESPUÉS de get_dummies (primeras 5 filas y algunas columnas relevantes):
   #        Name  HP  Attack  Defense  Sp. Atk  Sp. Def  Speed  Type1_Bug  \
0  1   Bulbasaur  45      49       49       65       65     45      False   
1  2     Ivysaur  60      62       63       80       80     60      False   
2  3    Venusaur  80      82       83      100      100     80      False   
3  4  Charmander  39      52       43       60       50     65      False   
4  5  Charmeleon  58      64       58       80       65     80      False   

   Type1_Dark  ...  Type2_Ghost  Type2_Grass  Type2_Ground  Type2_Ice  \
0       False  ...        False        False         False      False   
1       False  ...        False        False         False      False   
2       False  ...        False        False         False      False   
3       False  ...        Fal

### # Los Tipos se Convierten en Vectores Binarios 

**¿Entonces los tipos pasan a ser vectores $[0, 0, 1]$ por ejemplo?**

¡Exactamente! Con la Codificación One-Hot, cada categoría única de una variable (como 'Water', 'Fire', 'Grass' en el caso de los tipos de Pokémon) se transforma en una dimensión en un nuevo espacio vectorial. Para cada Pokémon (o fila en tu DataFrame), su `Type 1` y `Type 2` (si lo tiene) se representarán como un vector binario.

Tomemos el ejemplo de los tipos y asumamos que, después de la codificación One-Hot, tienes las siguientes columnas:

| Columna de Tipo |
|-----------------|
| `Type1_Water`   |
| `Type1_Fire`    |
| `Type1_Grass`   |
| `Type2_Flying`  |
| `Type2_Poison`  |
| `Type2_None`    |
| ...             |

Si un Pokémon es de `Type 1: Water` y `Type 2: None`, su parte del vector de características relacionada con los tipos se vería así:

$$ \text{Pokémon Vector (fragmento de tipos)} = [1, 0, 0, 0, 0, 1, \dots] $$

* `Type1_Water`: 1
* `Type1_Fire`: 0
* `Type1_Grass`: 0
* `Type2_Flying`: 0
* `Type2_Poison`: 0
* `Type2_None`: 1
* ... y así sucesivamente para todos los tipos posibles.

Si otro Pokémon es de `Type 1: Fire` y `Type 2: Flying`:

$$ \text{Pokémon Vector (fragmento de tipos)} = [0, 1, 0, 1, 0, 0, \dots] $$

Como puedes ver, cada Pokémon se convierte en un punto (un vector) en un espacio de muchas dimensiones, donde cada dimensión representa una categoría de tipo posible.

---

**¿Se pueden realizar operaciones con ellos después, por ejemplo?**

¡Sí, y de hecho, es precisamente para eso! La conversión de categorías a vectores numéricos es el paso clave que permite realizar operaciones matemáticas que son el corazón de muchos algoritmos de Machine Learning, incluyendo la **similitud del coseno** que usas en tu recomendador.

Una vez que los tipos se han transformado en estas representaciones vectoriales binarias (y las estadísticas numéricas se han escalado y añadido a estos mismos vectores), cada Pokémon y las preferencias del usuario se convierten en un **vector de características completo**.

Con estos vectores, puedes realizar operaciones como:

1.  **Producto Punto (Dot Product):** Es la base de la similitud del coseno. El producto punto entre dos vectores te da una medida de cuánto "se alinean" en el espacio multidimensional. Para vectores binarios, el producto punto simplemente cuenta el número de características compartidas (donde ambos vectores tienen un '1').
    * **Fórmula:** $A \cdot B = \sum_{i=1}^{n} A_i B_i$
    * **Ejemplo:** Si el usuario quiere `Type1_Water` y `Type2_None`, y un Pokémon es `Type1_Water` y `Type2_None`, el producto punto en las dimensiones de tipo será alto porque ambos tienen un '1' en las mismas columnas. Si el Pokémon es `Type1_Fire` y `Type2_Flying`, el producto punto en las dimensiones de tipo será 0 porque no comparten ningún '1' en las mismas columnas de tipo.

2.  **Cálculo de Magnitud (Norma Euclidiana):**
    * **Fórmula:** $||A|| = \sqrt{\sum_{i=1}^{n} A_i^2}$
    * Esto es la "longitud" del vector. En el caso de vectores One-Hot, esto simplemente es la raíz cuadrada del número de "1"s en el vector.

3.  **Similitud del Coseno:** Combina el producto punto y las magnitudes.
    * **Fórmula:** $\text{similitud}(A, B) = \frac{A \cdot B}{||A|| \cdot ||B||}$
    * Esta operación calcula el coseno del ángulo entre los dos vectores. Un ángulo más pequeño (coseno cercano a 1) indica mayor similitud. Un ángulo de 90 grados (coseno de 0) indica ortogonalidad (no relacionados).

**En resumen:**

La codificación One-Hot convierte la información categórica (como los tipos) en un formato numérico vectorial. Este formato es universalmente entendido por los algoritmos numéricos, lo que permite realizar operaciones matemáticas como el producto punto y el cálculo de la magnitud, que a su vez son los componentes esenciales para medir la **similitud (distancia)** entre Pokémon o entre las preferencias del usuario y un Pokémon. De esta manera, el sistema puede cuantificar cuán "parecido" es un Pokémon a lo que el usuario está buscando, tanto en términos de tipo como de estadísticas.

### # Normalización (Escalado Estándar - `StandardScaler`)

**1. Teoría: ¿Qué es la Normalización y por qué es fundamental?**

**Concepto:**

La Normalización (o, más específicamente en este caso, la **Estandarización Z-score**) es una técnica de preprocesamiento de datos utilizada para transformar las características numéricas de un dataset. El objetivo principal es llevar todas estas características a una escala común, sin distorsionar las diferencias en los rangos de valores ni la distribución de las características individuales.

El `StandardScaler` es una implementación específica de esta técnica que transforma los datos de modo que tengan una **media (promedio) de 0** y una **desviación estándar de 1**.

**El Problema de la Escala en los Datos:**

Considera las estadísticas base de los Pokémon:
* **HP:** Puede variar desde unos 1 HP hasta más de 250 HP (ej., Blissey, Chansey).
* **Speed:** Puede variar desde unos pocos puntos hasta más de 150 (ej., Deoxys-Speed, Ninjask).

Si comparamos directamente un HP de 200 con una Speed de 100, el valor de HP es el doble de grande. Sin normalización, un algoritmo que calcule distancias o similitudes (como la similitud del coseno) podría percibir que la diferencia en HP es mucho más significativa que la diferencia en Speed, simplemente porque los números son mayores.

Esto es problemático si todas las características tienen una importancia similar en la decisión del modelo, pero sus rangos de valores son muy diferentes. Las características con valores absolutos mayores dominarían el cálculo de la similitud o la distancia, y las características con valores más pequeños tendrían un impacto desproporcionadamente menor.

**La Solución de la Estandarización:**

La estandarización resuelve esto centrando los datos alrededor de cero y escalándolos por su desviación estándar. Después de la estandarización:

* **Media Cero:** La media de cada característica será 0.
* **Varianza Unitaria (Desviación Estándar de 1):** La desviación estándar de cada característica será 1.

Esto no solo hace que todas las características numéricas estén en una escala comparable, sino que también ayuda a que muchos algoritmos de Machine Learning converjan más rápido y funcionen de manera más efectiva, ya que evitan problemas numéricos derivados de rangos de valores muy dispares.

**Fórmula Matemática:**

Para cada valor individual $x$ en una característica (columna), su valor estandarizado $z$ se calcula utilizando la siguiente fórmula:

$$z = \frac{x - \mu}{\sigma}$$

Donde:
* $x$: Es el valor original de un dato específico en la característica.
* $\mu$ (mu): Es la media (promedio) de **todos los valores** de esa característica en el conjunto de datos de entrenamiento.
* $\sigma$ (sigma): Es la desviación estándar de **todos los valores** de esa característica en el conjunto de datos de entrenamiento.

**Ejemplo Teórico:**

Si tenemos la siguiente pequeña muestra de HP: `[100, 120, 80]`
* Media ($\mu$): $(100 + 120 + 80) / 3 = 100$
* Desviación Estándar ($\sigma$): (aproximadamente) $16.33$

* Para $x = 100$: $z = (100 - 100) / 16.33 = 0$
* Para $x = 120$: $z = (120 - 100) / 16.33 \approx 1.22$
* Para $x = 80$: $z = (80 - 100) / 16.33 \approx -1.22$

Los valores originales de `HP` ahora están centrados alrededor de 0, y su dispersión se mide en unidades de desviación estándar. Un valor de HP que era exactamente la media original, ahora es 0. Un HP 1.22 desviaciones estándar por encima de la media, ahora es 1.22.

---

In [4]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Suponemos que 'df' ya ha sido cargado y preprocesado (fillna y get_dummies)
# df_processed es el DataFrame que ya tiene las columnas de tipos One-Hot
# Ejemplo de cómo se vería df_processed antes de escalar las columnas numéricas:
#    #    Name  HP  Attack  Defense  Sp. Atk  Sp. Def  Speed  Type1_Bug  Type1_Dark  ...  Type2_None  Type2_Poison  Type2_Psychic
# 0   1  Bulbasaur  45      49       49       65       65     45          0           0  ...           0             1              0
# 1   2  Ivysaur    60      62       63       80       80     60          0           0  ...           0             1              0

# Definir las columnas numéricas a escalar
numeric_cols = ['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']

# --- Parte del código para la normalización (escalado estándar) ---

# 1. Crear una instancia del StandardScaler
scaler = StandardScaler()

# 2. Ajustar el escalador a los datos y transformar las columnas numéricas
# 'fit()' calcula la media y la desviación estándar para cada columna.
# 'transform()' aplica la fórmula (x - mu) / sigma usando esos valores.
# 'fit_transform()' hace ambos pasos en uno.
df_processed[numeric_cols] = scaler.fit_transform(df_processed[numeric_cols])

# --- Verificamos el resultado después del escalado ---
# Puedes ejecutar esto en tu Jupyter Notebook para ver cómo cambiaron los valores numéricos
print("\nEstadísticas numéricas DESPUÉS de StandardScaler (primeras 5 filas):")
print(df_processed[numeric_cols].head())
print("\nMedia de las columnas numéricas DESPUÉS de StandardScaler (deberían ser cercanas a 0):")
print(df_processed[numeric_cols].mean())
print("\nDesviación estándar de las columnas numéricas DESPUÉS de StandardScaler (deberían ser cercanas a 1):")
print(df_processed[numeric_cols].std())


Estadísticas numéricas DESPUÉS de StandardScaler (primeras 5 filas):
         HP    Attack   Defense   Sp. Atk   Sp. Def     Speed
0 -0.824982 -0.840697 -0.688238 -0.104674 -0.149276 -0.751585
1 -0.280565 -0.394557 -0.231793  0.421899  0.389211 -0.200035
2  0.445325  0.291811  0.420270  1.123996  1.107194  0.535366
3 -1.042749 -0.737742 -0.883857 -0.280198 -0.687764 -0.016185
4 -0.353154 -0.325921 -0.394809  0.421899 -0.149276  0.535366

Media de las columnas numéricas DESPUÉS de StandardScaler (deberían ser cercanas a 0):
HP         2.017768e-16
Attack     2.450147e-16
Defense    1.873642e-16
Sp. Atk   -1.008884e-16
Sp. Def    2.161895e-17
Speed     -1.603405e-16
dtype: float64

Desviación estándar de las columnas numéricas DESPUÉS de StandardScaler (deberían ser cercanas a 1):
HP         1.001016
Attack     1.001016
Defense    1.001016
Sp. Atk    1.001016
Sp. Def    1.001016
Speed      1.001016
dtype: float64


### # Desestandarización de Valores y su Aplicación en tu Proyecto

**1. Teoría: ¿Es necesaria la desestandarización y cómo se aplica en este caso?**

**Concepto de Desestandarización:**

La **desestandarización** es el proceso inverso a la estandarización (o normalización). Consiste en transformar los datos que han sido escalados (por ejemplo, con `StandardScaler`) de nuevo a su escala original, antes de que se les aplicara cualquier transformación.

Si la fórmula para estandarizar un valor $x$ a $z$ es:

$$z = \frac{x - \mu}{\sigma}$$

Entonces, la fórmula para desestandarizar un valor $z$ de vuelta a su escala original $x$ es:

$$x = z \cdot \sigma + \mu$$

Donde:
* $x$: Es el valor original de la característica.
* $z$: Es el valor estandarizado.
* $\mu$: Es la media de la característica que se usó durante la estandarización.
* $\sigma$: Es la desviación estándar de la característica que se usó durante la estandarización.

**¿Cuándo es crucial desestandarizar?**

La desestandarización es fundamental cuando la **interpretación de los resultados por parte de un humano o la aplicación de estos resultados en un contexto del mundo real requiere que los valores estén en su escala original.**

* **Ejemplos Típicos:**
    * Si un modelo de Machine Learning predice el **precio de una casa**, querrás el precio en dólares (ej., $350,000), no un valor estandarizado como `1.2`.
    * Si pronosticas las **ventas de un producto**, necesitas el número real de unidades (ej., 500 unidades), no un valor `0.8` estandarizado.
    * Si los coeficientes de un modelo lineal se interpretan directamente en relación con la escala de las características.

**¿Cuándo NO es estrictamente necesaria en el contexto del algoritmo?**

Para los algoritmos de Machine Learning que se basan en **distancias o similitudes** (como la similitud del coseno en tu recomendador), la estandarización de los datos es, de hecho, **esencial** para que el algoritmo funcione correctamente.

* El cálculo de la similitud del coseno (o la distancia euclidiana, etc.) se realiza directamente sobre los valores estandarizados. Revertir la estandarización antes de este cálculo anularía el propósito de haber estandarizado los datos, ya que las características con rangos más amplios volverían a dominar la medida de similitud.
* El resultado de la similitud (un valor entre -1 y 1 para el coseno) ya es una métrica significativa en sí misma y no requiere desestandarización.

**¿Cómo se maneja la "desestandarización" en tu proyecto de recomendación?**

Tu intuición es parcialmente correcta: para la presentación de los resultados al usuario, los valores deben ser comprensibles. Sin embargo, el método que utilizas para lograr esto es elegante y común en el desarrollo de sistemas de recomendación: **no necesitas un paso explícito de desestandarización de las características de los Pokémon, porque nunca estandarizaste el DataFrame original del que recuperas la información para mostrar.**

* **Para el cálculo de la similitud:** Las estadísticas del usuario y las de los Pokémon en `df_processed` (la copia que se usa para el cálculo) *sí* están estandarizadas. Esto garantiza que la similitud del coseno funcione de manera justa para todas las estadísticas y tipos.
* **Para la presentación al usuario:** Cuando seleccionas los Pokémon recomendados para mostrarlos en la interfaz de Streamlit (en la tabla Markdown y en el gráfico de barras), **obtienes los datos directamente del DataFrame original (`df`)**, que contiene las estadísticas de HP, Attack, Defense, etc., en sus valores naturales (sin escalar).

De esta manera, el algoritmo de similitud trabaja con datos óptimamente escalados, mientras que la interfaz de usuario presenta información en un formato que es directamente comprensible para el usuario, sin necesidad de un paso de desestandarización intermedio explícito para los Pokémon encontrados. La desestandarización explícita con `scaler.inverse_transform()` solo sería necesaria si, por alguna razón, el resultado de tu modelo fuera un valor estandarizado que necesitaras convertir de nuevo a la escala original para su uso o visualización.

---

### # Cómo se Mantienen las Referencias entre DataFrames (El Rol del Índice)

**Tu pregunta:** "Si usas los valores del DataFrame original, ¿cómo sabes si los que recomendaste son exactamente los que están en el DataFrame original?"

Esta es una excelente pregunta y la respuesta radica en cómo Pandas maneja los datos y, específicamente, en el **índice del DataFrame**.

**El Concepto Clave: El Índice del DataFrame**

Cada fila en un DataFrame de Pandas tiene una etiqueta única, llamada **índice**. Si no especificas un índice al cargar los datos, Pandas crea uno numérico por defecto, comenzando desde 0 y aumentando en 1 para cada fila (0, 1, 2, ...). Este índice es crucial porque actúa como un identificador único para cada fila de datos.

Imagina el índice como el "número de identificación" de cada Pokémon dentro de tu DataFrame.

**Cómo Opera el Proceso en tu Código:**

1.  **Carga del DataFrame Original (`df`):**
    Cuando cargas `pkmn.csv` en `df`, Pandas le asigna un índice a cada fila. Por ejemplo, la fila 0 corresponde al primer Pokémon, la fila 1 al segundo, y así sucesivamente.

    ```python
    # df (DataFrame original)
    # index | # | Name      | Type 1 | Type 2 | HP | Attack | ...
    # ------|---|-----------|--------|--------|----|--------|-----
    # 0     | 1 | Bulbasaur | Grass  | Poison | 45 | 49     | ...
    # 1     | 2 | Ivysaur   | Grass  | Poison | 60 | 62     | ...
    # 2     | 3 | Venusaur  | Grass  | Poison | 80 | 82     | ...
    ```

2.  **Creación de `df_processed` (Copia y Transformación):**
    Cuando creas `df_processed = df.copy()` y luego aplicas `pd.get_dummies()` y `StandardScaler()`, es crucial que `df_processed` **mantenga el mismo índice que el `df` original**. Por defecto, `copy()` preserva el índice, y `pd.get_dummies()` y la asignación `df_processed[numeric_cols] = scaler.fit_transform(...)` también lo hacen.

    Así, si la fila con índice `0` era "Bulbasaur" en `df`, la fila con índice `0` en `df_processed` seguirá siendo "Bulbasaur", pero ahora con sus tipos codificados One-Hot y sus estadísticas estandarizadas.

    ```python
    # df_processed (DataFrame transformado)
    # index | # | Name      | HP_scaled | Attack_scaled | ... | Type1_Grass | Type1_Fire | Type2_Poison | Type2_None | ...
    # ------|---|-----------|-----------|---------------|-----|-------------|------------|--------------|------------|-----
    # 0     | 1 | Bulbasaur | -0.956    | -0.923        | ... | 1           | 0          | 1            | 0          | ...
    # 1     | 2 | Ivysaur   | -0.736    | -0.589        | ... | 1           | 0          | 1            | 0          | ...
    # 2     | 3 | Venusaur  | -0.449    | -0.198        | ... | 1           | 0          | 1            | 0          | ...
    ```

3.  **Cálculo de Similitud y Obtención de Índices de Similitud:**
    Cuando calculas `similarities = cosine_similarity(user_features, pokemon_features_df)`, `pokemon_features_df` es `df_processed.drop(columns=['Name', '#'], errors='ignore')`. Este DataFrame *también conserva los mismos índices* que `df_processed` y `df`.

    El resultado de `similarities.argsort()[0][::-1]` es una lista de los **índices de las filas** que corresponden a los Pokémon más similares, ordenados de forma descendente por similitud. Estos son los **mismos índices** que se usan en `df` y `df_processed`.

    Por ejemplo, si los índices 0, 1, y 2 son los más similares, `final_recommended_internal_indices` podría ser `[0, 1, 2]`.

    ```python
    # Paso en el código:
    # similar_pokemon_internal_indices = similarities.argsort()[0][::-1]
    # Supongamos que esto nos da algo como: [0, 1, 2, 5, 7, ...]
    # Y después de la lógica de priorización, final_recommended_internal_indices = [0, 1, 2]
    ```

4.  **Recuperación de los Datos para la Presentación (`df.loc[]`):**
    Aquí es donde la magia sucede. La línea:
    ```python
    recommended_pokemons = df.loc[final_recommended_internal_indices].copy()
    ```
    Utiliza el método `.loc[]` de Pandas. `.loc[]` permite seleccionar filas de un DataFrame basándose en sus etiquetas de índice.

    Como `final_recommended_internal_indices` contiene los índices de los Pokémon más similares (ej., `[0, 1, 2]`), `df.loc[[0, 1, 2]]` va al **DataFrame original (`df`)** y extrae las filas que tienen esos índices.

    ```python
    # df.loc[[0, 1, 2]]
    # index | # | Name      | Type 1 | Type 2 | HP | Attack | ...
    # ------|---|-----------|--------|--------|----|--------|-----
    # 0     | 1 | Bulbasaur | Grass  | Poison | 45 | 49     | ...
    # 1     | 2 | Ivysaur   | Grass  | Poison | 60 | 62     | ...
    # 2     | 3 | Venusaur  | Grass  | Poison | 80 | 82     | ...
    ```
    El resultado (`recommended_pokemons`) es un nuevo DataFrame que contiene todas las columnas (incluyendo las originales, no estandarizadas, de las estadísticas) para esos Pokémon específicos.

**En Resumen:**

Pandas es muy bueno manteniendo la correspondencia entre las filas de un DataFrame y su índice. Cuando procesas una copia de tu DataFrame (`df_processed`), los índices de las filas permanecen vinculados a los mismos Pokémon. El algoritmo de similitud te devuelve los **índices** de los Pokémon más relevantes. Luego, usas esos mismos índices para "mirar" en el **DataFrame original (`df`)** y obtener toda la información (incluidas las estadísticas no escaladas) de esos Pokémon específicos para la visualización.

Es como tener dos versiones de un catálogo de productos: una versión está muy detallada y codificada para un robot (tu `df_processed`), y la otra es la versión bonita y fácil de leer para el cliente (tu `df`). El robot te dice "dame el producto con ID 001, 002 y 003", y tú usas esos IDs para encontrar sus descripciones amigables en el catálogo para el cliente.

---